In [190]:
import importlib
import types
import copy
import os
import site
import distutils.sysconfig as sysconfig

In [ ]:
# import spacy
# from spacy import tokenizer
# from spacy.tokenizer import Tokenizer

In [69]:
lib_name = 'spacy'
lib = importlib.import_module(lib_name)

In [92]:
def recurse_getattr(base, list_attrs):
    
    attrs = copy.copy(list_attrs)
    
    if len(list_attrs) == 0: return base
    
    while(len(attrs) > 0):
        sub_attr = attrs.pop(0)
        try: 
            base = getattr(base, sub_attr)
        except:
            print(sub_attr)
            raise Exception('FailedGetAttr')
            
    return base    

#### Testing

In [93]:
list_objs = []
ret = recurse_getattr(lib, list_objs)
ret

<module 'spacy' from '/home/user/anaconda3/envs/devfastai/lib/python3.7/site-packages/spacy/__init__.py'>

In [94]:
list_objs = ['tokenizer']
ret = recurse_getattr(lib, list_objs)
ret

<module 'spacy.tokenizer' from '/home/user/anaconda3/envs/devfastai/lib/python3.7/site-packages/spacy/tokenizer.cpython-37m-x86_64-linux-gnu.so'>

In [95]:
list_objs = ['tokenizer', 'Tokenizer']
ret = recurse_getattr(lib, list_objs)
ret

spacy.tokenizer.Tokenizer

In [96]:
list_objs = ['crappy_input']
ret = recurse_getattr(lib, list_objs)
ret

crappy_input


Exception: FailedGetAttr

#### Setup

https://stackoverflow.com/questions/6463918/how-can-i-get-a-list-of-all-the-python-standard-library-modules

In [191]:
builtin_names = dir(__builtins__)

p = site.getsitepackages()
fns = os.listdir(p[0])
sitepkg_names = [fn.replace('.py','') for fn in fns 
                 if '-' not in fn]

std_lib = sysconfig.get_python_lib(standard_lib=True)
stdlib_names = [e.replace('.py','') for e in os.listdir(std_lib)] 

skip_names = builtin_names + sitepkg_names + stdlib_names

len(skip_names)

518

#### Main

In [235]:
q = 'ss'

In [236]:
q.lower()

'ss'

In [242]:
def walk_match(search_term, 
               lib, 
               b_case_sensitive=True,
               b_skip=True, 
               b_print=True,
               max_depth=7):
    
    global_match   = []
    clean_skip_names = [e for e in skip_names 
                        if search_term not in e]
    
    if b_case_sensitive:
        def case(s): return s
    else: 
        def case(s): return s.lower()
    
    def inner(list_objs, depth_counter=0):
        
        nonlocal global_match
        nonlocal clean_skip_names

        obj = recurse_getattr(lib, list_objs )

        def nonprivate(e):
            if len(e) < 1: return False
            return e[0] != '_'

        children = [e for e in dir(obj) if nonprivate(e)]

        for child in children:

            if b_skip:
                if child in clean_skip_names: 
                    continue

            sub_list_objs = copy.copy(list_objs)
            sub_list_objs += [child]

            if case(search_term) in case(child):
                global_match.append(sub_list_objs)
                if b_print: print(sub_list_objs)

            current = recurse_getattr(lib, sub_list_objs)

            if isinstance(current, types.ModuleType):
                if depth_counter < max_depth:
                    inner(sub_list_objs, depth_counter + 1)
    
    inner([], 0)
    
    return global_match

#### Examples

In [214]:
lib_name = 'spacy'
lib = importlib.import_module(lib_name)

In [215]:
%%time
matches = walk_match('token', lib, max_depth=7)

['lang', 'tokenizer_exceptions']
['pipeline', 'functions', 'merge_subtokens']
['pipeline', 'merge_subtokens']
['tokenizer']
['tokens']
['tokens', 'doc', 'Retokenizer']
['tokens', 'token']
CPU times: user 2.31 s, sys: 15.6 ms, total: 2.33 s
Wall time: 2.3 s


In [216]:
%%time
matches = walk_match('token', lib, max_depth=8)

['lang', 'tokenizer_exceptions']
['pipeline', 'functions', 'merge_subtokens']
['pipeline', 'merge_subtokens']
['tokenizer']
['tokens']
['tokens', 'doc', 'Retokenizer']
['tokens', 'token']
CPU times: user 9.55 s, sys: 0 ns, total: 9.55 s
Wall time: 9.63 s


#### Show Case Sensitive ON

In [233]:
lib_name = 'torchvision'
lib = importlib.import_module(lib_name)
matches = walk_match('cnn', lib)

['models', 'detection', 'faster_rcnn']
['models', 'detection', 'faster_rcnn', 'fasterrcnn_resnet50_fpn']
['models', 'detection', 'fasterrcnn_resnet50_fpn']
['models', 'detection', 'generalized_rcnn']
['models', 'detection', 'keypoint_rcnn']
['models', 'detection', 'keypoint_rcnn', 'keypointrcnn_resnet50_fpn']
['models', 'detection', 'keypointrcnn_resnet50_fpn']
['models', 'detection', 'mask_rcnn']
['models', 'detection', 'mask_rcnn', 'maskrcnn_resnet50_fpn']
['models', 'detection', 'maskrcnn_resnet50_fpn']
['models', 'detection', 'roi_heads', 'fastrcnn_loss']
['models', 'detection', 'roi_heads', 'keypointrcnn_inference']
['models', 'detection', 'roi_heads', 'keypointrcnn_loss']
['models', 'detection', 'roi_heads', 'maskrcnn_inference']
['models', 'detection', 'roi_heads', 'maskrcnn_loss']


In [234]:
lib_name = 'torchvision'
lib = importlib.import_module(lib_name)
matches = walk_match('CNN', lib)

['models', 'detection', 'FasterRCNN']
['models', 'detection', 'KeypointRCNN']
['models', 'detection', 'MaskRCNN']
['models', 'detection', 'faster_rcnn', 'FastRCNNPredictor']
['models', 'detection', 'faster_rcnn', 'FasterRCNN']
['models', 'detection', 'faster_rcnn', 'GeneralizedRCNN']
['models', 'detection', 'faster_rcnn', 'GeneralizedRCNNTransform']
['models', 'detection', 'generalized_rcnn', 'GeneralizedRCNN']
['models', 'detection', 'keypoint_rcnn', 'FasterRCNN']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNN']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNNHeads']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNNPredictor']
['models', 'detection', 'mask_rcnn', 'FasterRCNN']
['models', 'detection', 'mask_rcnn', 'MaskRCNN']
['models', 'detection', 'mask_rcnn', 'MaskRCNNHeads']
['models', 'detection', 'mask_rcnn', 'MaskRCNNPredictor']
['models', 'detection', 'transform', 'GeneralizedRCNNTransform']


#### Show Case Sensitive OFF

In [243]:
lib_name = 'torchvision'
lib = importlib.import_module(lib_name)
matches = walk_match('cnn', lib, b_case_sensitive=False)

['models', 'detection', 'FasterRCNN']
['models', 'detection', 'KeypointRCNN']
['models', 'detection', 'MaskRCNN']
['models', 'detection', 'faster_rcnn']
['models', 'detection', 'faster_rcnn', 'FastRCNNPredictor']
['models', 'detection', 'faster_rcnn', 'FasterRCNN']
['models', 'detection', 'faster_rcnn', 'GeneralizedRCNN']
['models', 'detection', 'faster_rcnn', 'GeneralizedRCNNTransform']
['models', 'detection', 'faster_rcnn', 'fasterrcnn_resnet50_fpn']
['models', 'detection', 'fasterrcnn_resnet50_fpn']
['models', 'detection', 'generalized_rcnn']
['models', 'detection', 'generalized_rcnn', 'GeneralizedRCNN']
['models', 'detection', 'keypoint_rcnn']
['models', 'detection', 'keypoint_rcnn', 'FasterRCNN']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNN']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNNHeads']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNNPredictor']
['models', 'detection', 'keypoint_rcnn', 'keypointrcnn_resnet50_fpn']
['models', 'detection', 'keypointr

In [244]:
matches = walk_match('keypoint', lib, b_case_sensitive=False)

['models', 'detection', 'KeypointRCNN']
['models', 'detection', 'keypoint_rcnn']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNN']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNNHeads']
['models', 'detection', 'keypoint_rcnn', 'KeypointRCNNPredictor']
['models', 'detection', 'keypoint_rcnn', 'keypointrcnn_resnet50_fpn']
['models', 'detection', 'keypointrcnn_resnet50_fpn']
['models', 'detection', 'roi_heads', 'heatmaps_to_keypoints']
['models', 'detection', 'roi_heads', 'keypointrcnn_inference']
['models', 'detection', 'roi_heads', 'keypointrcnn_loss']
['models', 'detection', 'roi_heads', 'keypoints_to_heatmap']
['models', 'detection', 'transform', 'resize_keypoints']


#### We can't avoid aliases like `np` doe

In [220]:
lib_name = 'sklearn'
lib = importlib.import_module(lib_name)

In [222]:
matches = walk_match('r2', lib, max_depth=6)

['base', 'np', 'polynomial', 'chebyshev', 'chebvander2d']
['base', 'np', 'polynomial', 'hermite', 'hermvander2d']
['base', 'np', 'polynomial', 'hermite_e', 'hermevander2d']
['base', 'np', 'polynomial', 'laguerre', 'lagvander2d']
['base', 'np', 'polynomial', 'legendre', 'legvander2d']
['base', 'np', 'polynomial', 'polynomial', 'polyvander2d']
['metrics', 'pairwise', 'distance', 'np', 'polynomial', 'chebyshev', 'chebvander2d']
['metrics', 'pairwise', 'distance', 'np', 'polynomial', 'hermite', 'hermvander2d']
['metrics', 'pairwise', 'distance', 'np', 'polynomial', 'hermite_e', 'hermevander2d']
['metrics', 'pairwise', 'distance', 'np', 'polynomial', 'laguerre', 'lagvander2d']
['metrics', 'pairwise', 'distance', 'np', 'polynomial', 'legendre', 'legvander2d']
['metrics', 'pairwise', 'distance', 'np', 'polynomial', 'polynomial', 'polyvander2d']
['metrics', 'pairwise', 'np', 'polynomial', 'chebyshev', 'chebvander2d']
['metrics', 'pairwise', 'np', 'polynomial', 'hermite', 'hermvander2d']
['metr

In [221]:
matches = walk_match('score', lib, max_depth=6)

['metrics', 'accuracy_score']
['metrics', 'adjusted_mutual_info_score']
['metrics', 'adjusted_rand_score']
['metrics', 'average_precision_score']
['metrics', 'balanced_accuracy_score']
['metrics', 'brier_score_loss']
['metrics', 'calinski_harabasz_score']
['metrics', 'cluster', 'adjusted_mutual_info_score']
['metrics', 'cluster', 'adjusted_rand_score']
['metrics', 'cluster', 'calinski_harabasz_score']
['metrics', 'cluster', 'completeness_score']
['metrics', 'cluster', 'consensus_score']
['metrics', 'cluster', 'davies_bouldin_score']
['metrics', 'cluster', 'fowlkes_mallows_score']
['metrics', 'cluster', 'homogeneity_score']
['metrics', 'cluster', 'mutual_info_score']
['metrics', 'cluster', 'normalized_mutual_info_score']
['metrics', 'cluster', 'silhouette_score']
['metrics', 'cluster', 'v_measure_score']
['metrics', 'cohen_kappa_score']
['metrics', 'completeness_score']
['metrics', 'consensus_score']
['metrics', 'davies_bouldin_score']
['metrics', 'dcg_score']
['metrics', 'explained_var